In [1]:
# Import necessary modules
# glob allows unix style pathname pattern expansion
from glob import glob
# pydicom is the python dicom reader
import pydicom as dicom
import nibabel.nicom.csareader as csareader
import os
import pandas as pd

/tmp/ipykernel_125995/1246310376.py:6: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  import nibabel.nicom.csareader as csareader


In [26]:
# 5-T1w_MPR_vNav -- MPRAGE structrual dicoms
# 6-fMRI_DistortionMap_PA -- fmri field map dicoms
# 7-fMRI_DistortionMap_AP -- fmri field map dicoms opposite phase encode direction
# 9-fMRI_REVL_ROI_loc_2 -- fmri localizer dicoms run1
# 10-fMRI_REVL_Study_1 -- fmri task dicoms run1
# 14-dMRI_DistortionMap_AP_dMRI_REVL -- dwi field map dicoms
# 16-dMRI_AP_REVL -- diffusion weighted dicoms

user_ad = 'amattfel'
# directories will have to be specific to your computer
base_dir = f'/home/{user_ad}/Mattfeld_PSB6351'
proj_dir = '/sourcedata'

In [43]:
# Elements we're interested in
# Repetition Time
# Echo Time
# Acquisition Matrix 
# Flip Angle
# Acquisition Number
# Protocol Name
# Slice times = data_set[0x00191029].value

scans_of_interest = ['5-', '6-', '7-', '9-', '10-', '14-', '16-']

for curr_scan in scans_of_interest:
    
    subj_dir = f'/Mattfeld_REVL-000-vCAT-021-S1/scans/{curr_scan}*/*/DICOM/*'

    # with strings I can concatenate them with simple addition lines
    data_dir = base_dir + proj_dir + subj_dir

    # Here I use glob to grab the dicom files
    # Why would I use glob?  What does it give me?
    data_files = sorted(glob(data_dir + '/*'))

    print('')
    print('')
    print(data_files[0].split('/')[7])

    mri_vol = dicom.dcmread(data_files[0])
    
    print('The number of TRs collected equals: {0}'.format(len(data_files)))
    print('The timing of the TR was: {0} ms'.format(mri_vol.RepetitionTime))
    print('The timing of the TE was: {0} ms'.format(mri_vol.EchoTime))
    print(f'The flip angle in degrees was: {mri_vol.FlipAngle}')
    print(f'The name of the scan was: {mri_vol.ProtocolName}')
    pe_dir = mri_vol[int('00181312', 16)].value
    print('The phase encode direction was: {0}'.format(mri_vol[int('00181312', 16)].value))
    print('COL = AP or PA; ROW = RL or LR')
    if 0x00191029 in mri_vol:
        print(f'The slice timing was: {mri_vol[0x00191029].value}')
    print(f'The field of view - FOV was: {mri_vol[0x0051100c].value}')

    # Why use the following format to access information in the header?
    #print(data_set[int('0051100c', 16)].value) # data_set[0x0051100c].value
    # Allows us to access information that is private in the dicom header.
    
    # How do I find phase encoding direction information?
    
    csa_str = mri_vol[int('00291010', 16)].value
    csa_tr = csareader.read(csa_str)
    pe_sign = csa_tr['tags']['PhaseEncodingDirectionPositive']['items'][0]
    print('The phase encode direction was: {0}'.format(csa_tr['tags']['PhaseEncodingDirectionPositive']['items'][0]))
    print('1 = +; 0 = -')
    print('COL, 1 = PA; COL, 0 = AP')
    
    rowcol_to_niftidim = {'COL': 'i', 'ROW': 'j'}
    pedp_to_sign = {0: '-', 1: ''} 
    ij = rowcol_to_niftidim[pe_dir]
    sign = pedp_to_sign[pe_sign]
    print(f'The phase encoding direction was: {sign}{ij}')





5-T1w_MPR_vNav
The number of TRs collected equals: 176
The timing of the TR was: 2500 ms
The timing of the TE was: 2.9 ms
The flip angle in degrees was: 8
The name of the scan was: T1w_MPR_vNav
The phase encode direction was: ROW
COL = AP or PA; ROW = RL or LR
The field of view - FOV was: FoV 256*256
The phase encode direction was: 1
1 = +; 0 = -
COL, 1 = PA; COL, 0 = AP
The phase encoding direction was: j


6-fMRI_DistortionMap_PA
The number of TRs collected equals: 66
The timing of the TR was: 6720 ms
The timing of the TE was: 80 ms
The flip angle in degrees was: 90
The name of the scan was: fMRI_DistortionMap_PA
The phase encode direction was: COL
COL = AP or PA; ROW = RL or LR
The field of view - FOV was: FoV 200*200
The phase encode direction was: 0
1 = +; 0 = -
COL, 1 = PA; COL, 0 = AP
The phase encoding direction was: -i


7-fMRI_DistortionMap_AP
The number of TRs collected equals: 66
The timing of the TR was: 6720 ms
The timing of the TE was: 80 ms
The flip angle in degrees w

In [32]:
func_vol.InPlanePhaseEncodingDirection

'COL'

In [45]:
func_vol[int('00200037', 16)].value #ImageOrientationPatientDICOM

[1, 0, 0, 0, 0.86602540210929, -0.5000000029014]

In [46]:
func_vol[int('00181312', 16)].value #InPlanePhaseEncodingDirectionDICOM
'''
For Siemens V* EPI axial acquisitions saved as mosaics we typically expect 0018,1312 to be COL for A>>P and P>>A 
or
j
'''

'COL'